In [1]:
import pandas as pd
import numpy as np
import os
import gc
from itertools import product
from tqdm import tqdm_notebook

import matplotlib.pyplot as plt
%matplotlib inline 

In [125]:
# Objective:
# predict total sales for every product and store in the next month based on lagged features

# NOTE:
# to use tqdm_notebook with python 3 (https://github.com/tqdm/tqdm/issues/187)

NOTE: score of 0.99291 which places team (BlackArrow) at rank 126 as of 3/13/2018
https://www.kaggle.com/c/competitive-data-science-final-project/leaderboard

### Load data

In [3]:
transactions = pd.read_csv('sales_train.csv.gz')
items = pd.read_csv('items.csv')
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')

print(transactions.shape)
print(items.shape)
print(item_categories.shape)
print(shops.shape)
transactions.head(3)

(2935849, 6)
(22170, 3)
(84, 2)
(60, 2)


,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.0,1.0
1,03.01.2013,0,25,2552,899.0,1.0
2,05.01.2013,0,25,2552,899.0,-1.0


### Create Feature Matrix

1. Downcast types to save memory (method taken from Advanced ML HSE)

In [4]:
def downcast_dtypes(df):
    ''' Changes column types in the dataframe: 
        `float64` type to `float32`
        `int64`   type to `int32`
    Args:
        df - pandas data frame
    Returns:
        df - downcasted data frame
    '''
    # Select columns to downcast
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]

    # Downcast
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols]   = df[int_cols].astype(np.int32)
    return df

def create_grid(sales, index_cols = ['shop_id', 'item_id', 'date_block_num']):
    # For every month we create a grid from all shops/items combinations from that month
    grid = [] 
    for block_num in sales['date_block_num'].unique():
        cur_shops = sales[sales['date_block_num']==block_num]['shop_id'].unique()
        cur_items = sales[sales['date_block_num']==block_num]['item_id'].unique()
        grid.append(np.array(list(product(*[cur_shops, cur_items, [block_num]])),dtype='int32'))

    # Turn the grid into pandas dataframe
    grid = pd.DataFrame(np.vstack(grid), columns = index_cols,dtype=np.int32)

    # Get aggregated values for (shop_id, item_id, month)
    gb = sales.groupby(index_cols,as_index=False).agg({'item_cnt_day':{'target':'sum'}})
    gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
    
    # Join aggregated data to the grid
    all_data = pd.merge(grid,gb,how='left',on=index_cols).fillna(0)
    
    # shop-month aggregates
    gb = sales.groupby(['shop_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_shop':'sum'}})
    gb.columns = [col[0] if col[-1]=='' else col[-1] for col in gb.columns.values]
    all_data = pd.merge(all_data, gb, how='left', on=['shop_id', 'date_block_num']).fillna(0)

    # item-month aggregates
    gb = sales.groupby(['item_id', 'date_block_num'],as_index=False).agg({'item_cnt_day':{'target_item':'sum'}})
    gb.columns = [col[0] if col[-1] == '' else col[-1] for col in gb.columns.values]
    all_data = pd.merge(all_data, gb, how='left', on=['item_id', 'date_block_num']).fillna(0)
    
    # Sort the data
    all_data.sort_values(['date_block_num','shop_id','item_id'],inplace=True)
    all_data = downcast_dtypes(all_data)
    return all_data

In [5]:
path = "transactions_all_data_2-1.csv"
# Create "grid" with columns
index_cols = ['shop_id', 'item_id', 'date_block_num']

if os.path.isfile(path):
    data = pd.read_csv(path)
else:
    data = create_grid(transactions, index_cols)
    data.to_csv(path, index=False)
    
gc.collect();

In [6]:
data.head()

,shop_id,item_id,date_block_num,target,target_shop,target_item
0,0,19,0,0.0,5578.0,1.0
1,0,27,0,0.0,5578.0,7.0
2,0,28,0,0.0,5578.0,8.0
3,0,29,0,0.0,5578.0,4.0
4,0,32,0,6.0,5578.0,299.0


In [42]:
data.shape

(10913850, 6)

### Aggregates as a form of mean encoding
Note that assigning aggregate values as a feature is a form of mean encoding.

In [52]:
# Check if aggregates make sense
# Total sales for item_id 32 for date block 0 is 299
assert(data[(data.item_id == 32) & (data.date_block_num==0)].target.sum()==299)
# Total sales for shop_id 0 for date block 0 is 5578
assert(data[(data.shop_id == 0) & (data.date_block_num==0)].target.sum()==5578)


#### Change test set dimensions to match the rest of data

In [48]:
test = pd.read_csv('test.csv')
test.head(3)

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233


In [49]:
test_orig = test.copy()
test['date_block_num'] = np.ones(len(test.index))* 34
test['target'] = np.zeros(len(test.index))
test['target_shop'] = np.zeros(len(test.index))
test['target_item'] = np.zeros(len(test.index))
cols = test.columns.tolist()
print(cols)
test = test[cols]
ID = test['ID']
del test['ID']

test.head()

['ID', 'shop_id', 'item_id', 'date_block_num', 'target', 'target_shop', 'target_item']


,shop_id,item_id,date_block_num,target,target_shop,target_item
0,5,5037,34.0,0.0,0.0,0.0
1,5,5320,34.0,0.0,0.0,0.0
2,5,5233,34.0,0.0,0.0,0.0
3,5,5232,34.0,0.0,0.0,0.0
4,5,5268,34.0,0.0,0.0,0.0


In [43]:
test.shape

(214200, 6)

In [39]:
all_data = pd.concat([data,test], axis=0)

In [44]:
assert(all_data.shape[0] == data.shape[0] + test.shape[0])

### Get Lagged features

Using 1,3,12 for ranges. (Note: Adding more lagged features resulted in memry issues. Need to address)

In [53]:
data = all_data


# List of columns that we will use to create lags
cols_to_rename = list(data.columns.difference(index_cols)) 
print(cols_to_rename)

# Lags to use
shift_range = [1,3,12]

for month_shift in tqdm_notebook(shift_range):
    
    train_shift = data[index_cols + cols_to_rename].copy()
    
    train_shift['date_block_num'] = train_shift['date_block_num'] + month_shift
    
    foo = lambda x: '{}_lag_{}'.format(x, month_shift) if x in cols_to_rename else x
    train_shift = train_shift.rename(columns=foo)

    data = pd.merge(data, train_shift, on=index_cols, how='left').fillna(0)

del train_shift

# Don't use old data from year 2013
data = data[data['date_block_num'] >= 12] 

# List of all lagged features
fit_cols = [col for col in data.columns if col[-1] in [str(item) for item in shift_range]] 
# We will drop these at fitting stage
to_drop_cols = list(set(list(data.columns)) - (set(fit_cols)|set(index_cols))) + ['date_block_num'] 

# Category for each item
item_category_mapping = items[['item_id','item_category_id']].drop_duplicates()

data = pd.merge(data, item_category_mapping, how='left', on='item_id')
data = downcast_dtypes(data)
gc.collect();

['target', 'target_item', 'target_shop']


HBox(children=(IntProgress(value=0, max=3), HTML(value='')))

In [54]:
path = "2-1-lagged-features-all.csv"

if os.path.isfile(path):
    data = pd.read_csv(path)
else:
    data.to_csv(path, index=False)

In [55]:
data.head()

,shop_id,item_id,date_block_num,target,target_shop,target_item,target_lag_1,target_item_lag_1,target_shop_lag_1,target_lag_3,target_item_lag_3,target_shop_lag_3,target_lag_12,target_item_lag_12,target_shop_lag_12,item_category_id
0,2,27,12.0,0.0,890.0,1.0,0.0,4.0,1322.0,0.0,6.0,795.0,1.0,7.0,1146.0,19
1,2,30,12.0,0.0,890.0,58.0,0.0,47.0,1322.0,0.0,24.0,795.0,0.0,0.0,0.0,40
2,2,31,12.0,0.0,890.0,15.0,0.0,25.0,1322.0,0.0,25.0,795.0,0.0,0.0,0.0,37
3,2,32,12.0,1.0,890.0,84.0,0.0,89.0,1322.0,0.0,58.0,795.0,0.0,299.0,1146.0,40
4,2,33,12.0,1.0,890.0,42.0,1.0,42.0,1322.0,0.0,33.0,795.0,1.0,61.0,1146.0,37


### Train/Valid/Test Split

In [69]:
# Save `date_block_num`, as we can't use them as features, but will need them to split the dataset into parts 
dates = data['date_block_num']

last_block = dates.max()
print('Test`date_block_num` is %d' % last_block)

Test`date_block_num` is 34


In [70]:
['target_item', 'target', 'target_shop', 'date_block_num']

['target_item', 'target', 'target_shop', 'date_block_num']

Split the data into train and validation.

Note: need to drop 'target_item', 'target', 'target_shop', 'date_block_num' as leaving them will introduce look ahead bias.

In [90]:
dates_train = dates[dates <  last_block]
dates_test  = dates[dates == last_block]
dates_valid = dates.unique()[-4:-1] # 31,32,33

# Drop columns that would introduce look ahead bias
X_train = data.loc[dates <  dates_valid[0]].drop(to_drop_cols, axis=1)
X_valid = data.loc[dates.isin(dates_valid)].drop(to_drop_cols, axis=1)
X_test =  data.loc[dates == last_block].drop(to_drop_cols, axis=1)

y_train = data.loc[dates < dates_valid[0], 'target'].values
y_valid = data.loc[dates.isin(dates_valid), 'target'].values
y_test =  data.loc[dates == last_block, 'target'].values



In [92]:
X_test.head()

,shop_id,item_id,target_lag_1,target_item_lag_1,target_shop_lag_1,target_lag_3,target_item_lag_3,target_shop_lag_3,item_category_id
6425094,5,5037,0.0,25.0,1052.0,3.0,119.0,1294.0,19
6425095,5,5320,0.0,0.0,0.0,0.0,0.0,0.0,55
6425096,5,5233,1.0,42.0,1052.0,1.0,150.0,1294.0,19
6425097,5,5232,0.0,28.0,1052.0,1.0,65.0,1294.0,23
6425098,5,5268,0.0,0.0,0.0,0.0,0.0,0.0,20


In [93]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

lr = LinearRegression()
lr.fit(X_train.values, y_train)
pred_lr = lr.predict(X_valid.values)

print('Test R-squared for linreg is %f' % r2_score(y_valid, pred_lr))

Test R-squared for linreg is 0.189216


In [94]:
import lightgbm as lgb

lgb_params = {
               'feature_fraction': 0.75,
               'metric': 'rmse',
               'nthread':1, 
               'min_data_in_leaf': 2**7, 
               'bagging_fraction': 0.75, 
               'learning_rate': 0.03, 
               'objective': 'mse', 
               'bagging_seed': 2**7, 
               'num_leaves': 2**7,
               'bagging_freq':1,
               'verbose':0 
              }

model = lgb.train(lgb_params, lgb.Dataset(X_train, label=y_train), 100)
pred_lgb = model.predict(X_valid)

print('Test R-squared for LightGBM is %f' % r2_score(y_valid, pred_lgb))

Test R-squared for LightGBM is 0.206673


In [96]:
# Create train data using all except test.
X_train_all = data.loc[dates <  last_block].drop(to_drop_cols, axis=1)
y_train_all = data.loc[dates < last_block, 'target'].values

In [97]:
model = lgb.train(lgb_params, lgb.Dataset(X_train_all, label=y_train_all), 100)
pred_test_lgb = model.predict(X_test)

In [98]:
pred_test_lgb

array([0.45806613, 0.13574635, 0.62028672, ..., 0.04939725, 0.04620208,
       0.04365277])

In [111]:
test_temp = test.copy()

In [114]:
test_temp.target = pred_test_lgb
del test_temp['target_shop']
del test_temp['target_item']
del test_temp['date_block_num']
test_temp.head()

,shop_id,item_id,target
0,5,5037,0.458066
1,5,5320,0.135746
2,5,5233,0.620287
3,5,5232,0.430201
4,5,5268,1.603025


In [120]:
submission = test_orig.copy()
submission_temp = submission.merge(test_temp, on=['shop_id','item_id'], how='left')
submission_temp.head()

,ID,shop_id,item_id,target
0,0,5,5037,0.458066
1,1,5,5320,0.135746
2,2,5,5233,0.620287
3,3,5,5232,0.430201
4,4,5,5268,1.603025


In [121]:
submission_temp.max()

ID         214199.000000
shop_id        59.000000
item_id     22167.000000
target        346.760877
dtype: float64

In [122]:
sample = pd.read_csv('sample_submission.csv')
final_submission = sample.copy()
final_submission.item_cnt_month = submission_temp.target.clip(0,20)
final_submission.head()

,ID,item_cnt_month
0,0,0.458066
1,1,0.135746
2,2,0.620287
3,3,0.430201
4,4,1.603025


In [123]:
final_submission.to_csv('2-1-v0-submission.csv',index=False)